# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [1]:
import  warnings
warnings.filterwarnings('ignore')
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np

import seaborn as sns
from IPython.display import display

In [2]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
df.head()

,admit,gre,gpa,prestige
0,0,380.0,3.61,3.0
1,1,660.0,3.67,3.0
2,1,800.0,4.00,1.0
3,1,640.0,3.19,4.0
4,0,520.0,2.93,4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [4]:
# frequency table for prestige and whether or not someone was admitted
gb = df.groupby(['prestige', 'admit']).size()
gb


prestige  admit
1.0       0        28
          1        33
2.0       0        95
          1        53
3.0       0        93
          1        28
4.0       0        55
          1        12
dtype: int64

## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [6]:
#create dummy variable for the prestige categorical variable
dummy_ranks = pd.get_dummies(df['prestige'], prefix='prestige')
dummy_ranks.head()
#Join the dummy variable to the main frame


,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0.0,0.0,1.0,0.0
1,0.0,0.0,1.0,0.0
2,1.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0
4,0.0,0.0,0.0,1.0


#### 2.2 When modeling our class variables, how many do we need? 



Answer: _If we have `n` types of different values in a single categorical feature, we need `n-1` dummy features, we need to rid-off one feature because they cause multicollinearity in the data set.So in this case we need 3_.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [7]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.loc[:, 'prestige_1':])
handCalc.head()

,admit,gre,gpa,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,0.0,0.0,1.0,0.0
1,1,660.0,3.67,0.0,0.0,1.0,0.0
2,1,800.0,4.00,1.0,0.0,0.0,0.0
3,1,640.0,3.19,0.0,0.0,0.0,1.0
4,0,520.0,2.93,0.0,0.0,0.0,1.0


In [8]:
#crosstab prestige 1 admission 
#frequency table cutting prestige and whether or not someone was admitted
pd.crosstab(handCalc['prestige_1.0'], handCalc['admit'])

admit,0,1
prestige_1.0,,
0.0,243,93
1.0,28,33


In [9]:
# frequency table cutting prestige and whether or not someone was admitted
pd.crosstab(df.prestige, df.admit, margins=True)

admit,0,1,All
prestige,,,
1.0,28,33,61
2.0,95,53,148
3.0,93,28,121
4.0,55,12,67
All,271,126,397


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [13]:
pd.crosstab(df.admit, handCalc['prestige_1.0'], margins=True)

prestige_1.0,0.0,1.0,All
admit,,,
0,243,28,271
1,93,33,126
All,336,61,397


#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [20]:
P_a = 33.0/61.0
P_n = 28/61.0

odds_Pa = (P_a/P_n)
odds_Pn = (P_n/P_a)

print P_n
#odds_admission_1 = odds_Pn/odds_Pa
#print odds_admission_1

#print 1/odds_admission_1

0.459016393443


#### 3.3 Calculate the odds ratio

In [19]:
P_a = 33.0/61.0
P_n = 28/61.0

odds_Pa = (P_a/P_n)
odds_Pn = (P_n/P_a)

odds_admission_1 = odds_Pa/odds_Pn
print odds_admission_1

#print 1/odds_admission_1

1.38903061224


#### 3.4 Write this finding in a sentenance: 

Answer: 
1. There is 45% probability that someone from prestige_1 university will not admit to UCLA.
2. Thus, for students who came from prestige_1, the odds of being admitted 1.38 larger than the odds being admiited from prestige_1, prestige_3, and prestige_4.

#### 3.5 Print the cross tab for prestige_4

In [23]:
pd.crosstab(handCalc['prestige_4.0'], df.admit, margins= True)

admit,0,1,All
prestige_4.0,,,
0.0,216,114,330
1.0,55,12,67
All,271,126,397


#### 3.6 Calculate the OR 

In [26]:
P_a = 12/67.0
P_n = 55/67.0

odds_Pa = (P_a/P_n)
odds_Pn = (P_n/P_a)

odds_admission_1 = odds_Pa/odds_Pn
print odds_admission_1

#print 1/odds_admission_1

0.0476033057851


#### 3.7 Write this finding in a sentence

Answer: There is 0.4% probability of admitting to UCLA who came from prestige_4

## Part 4. Analysis

In [27]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.loc[:, 'prestige_2':])
data.head()

,admit,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,0.0,1.0,0.0
1,1,660.0,3.67,0.0,1.0,0.0
2,1,800.0,4.00,0.0,0.0,0.0
3,1,640.0,3.19,0.0,0.0,1.0
4,0,520.0,2.93,0.0,0.0,1.0


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [29]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [30]:
train_cols = data.columns[1:]
train_cols


Index([u'gre', u'gpa', u'prestige_2.0', u'prestige_3.0', u'prestige_4.0',
       u'intercept'],
      dtype='object')

#### 4.2 Fit the model

In [33]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data[train_cols], data['admit'], test_size= 0.2, random_state=11)

lr = LogisticRegression(random_state=99).fit(data[train_cols], data['admit'])
"------------------------------------------------------------------------------------------------------------"

train_cols = data.columns[1:]
# Index([gre, gpa, prestige_2, prestige_3, prestige_4], dtype=object)

logit = sm.Logit(data['admit'], data[train_cols])

# fit the model
result = logit.fit()


Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [34]:
print result.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Sat, 16 Sep 2017   Pseudo R-squ.:                 0.08166
Time:                        16:41:13   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa              0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2.0    -0.6801      0.317     -2.14

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [35]:
#Take the exponential of each of the coefficients to generate the odds ratios. 
import numpy as np
print np.exp(result.params)

gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64


In [37]:
#confidence interval to get a better picture for how uncertainty in variables can impact the admission rate.
# odds ratios and 95% CI
params = result.params
conf = result.conf_int()
conf['OR'] = params
conf.columns = ['2.5%', '97.5%', 'OR']
np.exp(conf)

,2.5%,97.5%,OR
gre,1.000074,1.004372,1.002221
gpa,1.136120,4.183113,2.180027
prestige_2.0,0.272168,0.942767,0.506548
prestige_3.0,0.133377,0.515419,0.262192
prestige_4.0,0.093329,0.479411,0.211525
intercept,0.002207,0.194440,0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: 
_We can expect the odds of being admitted to decrease by about 50% if the prestige of a school is 2_.

#### 4.6 Interpret the OR of GPA

Answer: _The odds ratio of gpa tells us, there is 2.1 times larger for those who score gpa between 95% of CI in the gpa score. _

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [38]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [39]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [ ]:
#recreate the dummy variables
combos.columns= ['gre', 'gpa', 'prestige', 'intercept']
dummy_ranks = pd.get_dummies(combos['prestige'], prefix='prestige')
dummy_ranks.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']

# keep only what we need for making pre`dictions
cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = pd.concat([combos, dummy_ranks], axis=1)
#combos

#### 5.2 Make predictions on the enumerated dataset

In [58]:
train_cols

Index([u'gre', u'gpa', u'prestige_2.0', u'prestige_3.0', u'prestige_4.0',
       u'intercept'],
      dtype='object')

In [59]:
# make predictions on the enumerated dataset
#combos['admit_pred'] = result.predict(combos[[train_cols])
print combos.head()

     gre       gpa  prestige  intercept  prestige_2  prestige_3  prestige_4
0  220.0  2.260000       1.0        1.0         0.0         0.0         0.0
1  220.0  2.260000       2.0        1.0         1.0         0.0         0.0
2  220.0  2.260000       3.0        1.0         0.0         1.0         0.0
3  220.0  2.260000       4.0        1.0         0.0         0.0         1.0
4  220.0  2.453333       1.0        1.0         0.0         0.0         0.0


#### 5.3 Interpret findings for the last 4 observations

Answer: <br>
_As we look to the last 4 observations of data points, the students have in the same range od their GRE and GPA but that came from different prestige ranking schools. As we see in the above using odds ratios, the students came from prestige1 has more probability of jing to UCLA from others. In this case, the admission process is assessed based on the feature of prestige_ 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.